In [1]:
## Importing Libraries

# Python-based Libraries
from datetime import timedelta, datetime
from pathlib import Path
import os

# Data Analysis Libraries
import numpy as np
import pandas as pd

# Graph plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mplcols
from matplotlib import cm
import folium
import branca.colormap as bcm

    
# Filtering Data before Plotting


In [62]:
class Drifters:
    
    def __init__(self, name:str):
        self.name = name
        self.data = {}
        self.data_info = {}
        
    def read_data(self, path):
        # Define Possible encodings
        encodings = ["UTF-8", "UTF-16 LE"]

        # Check if path is a filepath or directory path
        if Path(path).is_file(): 
            # Get file name from path
            filename = path.split('/')[-1].split('.')[-2] 
            # Define a variable to track file decoding
            encoding_gotten = False
            for encoding_value in encodings:
                try:
                    data_value = pd.read_csv(path, encoding=encoding_value)
                    # Catch special errors whereby the encoding is wrong but the data is read.
                    if "Unnamed: 1" in data_value.columns: 
                        continue
                    encoding_gotten = True
                    break # Stop trying if the options had worked
                except:
                    pass # Continue trying if the option didn't work

            if encoding_gotten == True:
                data_to_keep = [['DeviceDateTime','Latitude','Longitude'], 
                            ['Position time (UTC)', 'Latitude (°)', 'Longitude (°)'],
                            ]
                for dtk_val in data_to_keep:
                    try:
                        # Select Data to Keep
                        data_value = data_value[dtk_val]
                        # Drop columns with missing values
                        data_value.dropna()
                        # Rename Header Columns
                        data_value.columns = ['DateTime','Latitude','Longitude']
                        # Convert DateTime
                        data_value['DateTime'] = pd.to_datetime(data_value['DateTime'])
                        # Store DF in the object instance 'data'
                        self.data[filename] = data_value
                        
                    except:
                        pass
            else:
                print(f"Encoding error. The encoding for {filename} data should be checked.")

        elif Path(path).is_dir():    
        # 1. Read the files in the given path
            file_names = os.listdir(path)

            # 2. Read each file into pandas dataframe
            for file in file_names:
                filepath = path+'/'+file
                filename = file.split('.')[-2] 
                # Define a variable to track file decoding
                encoding_gotten = False
                for encoding_value in encodings:
                    try:
                        data_value = pd.read_csv(filepath, encoding=encoding_value)
                        # Catch special errors whereby the encoding is wrong but the data is read.
                        if "Unnamed: 1" in data_value.columns: 
                            continue
                        encoding_gotten = True
                        break # Stop trying if the options had worked
                    except:
                        pass # Continue trying if the option didn't work

                # Store the data in the "data" dictionary
                if encoding_gotten == True:
                    data_to_keep = [['DeviceDateTime','Latitude','Longitude'], 
                               ['Position time (UTC)', 'Latitude (°)', 'Longitude (°)'],
                                ]
                    for dtk_val in data_to_keep:
                        try:
                            # Select Data to Keep
                            data_value = data_value[dtk_val]
                            # Drop columns with missing values
                            data_value.dropna(axis=0, inplace=True)
                            # Rename Header Columns
                            data_value.columns = ['DateTime','Latitude','Longitude']
                            # Convert DateTime
                            data_value['DateTime'] = pd.to_datetime(data_value['DateTime'])
                            # Store DF in the object instance 'data'
                            self.data[filename] = data_value
                        
                        except:
                            pass

                else:
                    print(f"Encoding error. The encoding for {filepath} data should be checked.")
    
    def read_logsheet(self, file_path):
        
        self.logsheet = pd.read_csv(file_path)
        
        self.logsheet.dropna(axis=0, inplace=True)
        
        self.logsheet["DepDateTime"] = self.logsheet.apply(lambda x: str(x["DepDate"]) + " " + str(x["DepTime"]), axis = 1)
        self.logsheet["RecovDateTime"] = self.logsheet.apply(lambda x: str(x["RecovDate"]) + " " + str(x["RecovTime"]), axis = 1)
        
        self.logsheet.drop(columns=["DepDate", "DepTime", "RecovDate", "RecovTime"], inplace=True)
        
        self.logsheet["DepDateTime"] = pd.to_datetime(self.logsheet["DepDateTime"])
        self.logsheet["RecovDateTime"] = pd.to_datetime(self.logsheet["RecovDateTime"])
        
        for key in self.data.keys():
            station = self.logsheet["Station"][self.logsheet["Name"] == float(key)].values
            drog_depth = self.logsheet["DrogDepth"][self.logsheet["Name"] == float(key)].values
            
            
            dep_datetime = self.logsheet["DepDateTime"][self.logsheet["Name"] == float(key)].values
            recov_datetime = self.logsheet["RecovDateTime"][self.logsheet["Name"] == float(key)].values
            
            dep_long = self.logsheet["DepLong"][self.logsheet["Name"] == float(key)].values
            dep_lat = self.logsheet["DepLat"][self.logsheet["Name"] == float(key)].values
            
            recov_long = self.logsheet["RecovLong"][self.logsheet["Name"] == float(key)].values
            recov_lat = self.logsheet["RecovLat"][self.logsheet["Name"] == float(key)].values
            
            
            info = {"Station":station, "DrogDepth":drog_depth, "DepDateTime":dep_datetime, "RecovDateTime":recov_datetime,
                    "DepLong":dep_long, "DepLat":dep_lat, "RecovLong":recov_long, "RecovLat":recov_lat}
            
            self.data_info[key] = info
    
    def time_shift(self, shift_amount:float = 1):
        for key in self.data.keys():
            self.data[key]["DateTime"] = self.data[key]["DateTime"] + timedelta(hours = shift_amount)
    
    def extract_data(self):
        for key in self.data_info.keys():
            # Get the start and end time for each experiment
            deploy_time = self.data_info[key]['DepDateTime']
            recov_time = self.data_info[key]['RecovDateTime']

            # Extract experiment data within the start and end time 
            # bool = np.array([(deploy_time < self.data[key]["DateTime"][i] < recov_time) for i in range(len(self.data[key]))]).flatten()
            bool = np.array([(deploy_time < pos_time < recov_time) for pos_time in self.data[key]["DateTime"]]).flatten()
            
            if bool.size != 0:
                self.data[key] = self.data[key][bool]
                # Drop columns with missing values
                self.data[key].dropna(axis=0, inplace=True)
                # Reset Index
                self.data[key].reset_index(inplace = True)
            
        
    
    def compute_velocity(self):
        R = 6373.0
        for key in self.data.keys():
            # data = self.data[key]
            # select columns for latitude, longtitude and time
            df_lat = np.array(self.data[key]["Latitude"])
            df_long = np.array(self.data[key]["Longitude"])
            df_time = self.data[key]["DateTime"]
            
            # compute delta time
            d_time = np.array(df_time[1:]) - df_time[:-1]
            d_time = d_time.apply(lambda dt_i : dt_i.seconds/60.0)
            
            # compute distance from lat,long
            rad_lat = np.radians(df_lat)
            rad_long = np.radians(df_long)
            d_lat = rad_lat[1:] - np.array(rad_lat[:-1])
            d_long = rad_long[1:] - np.array(rad_long[:-1]) 
            
            a = np.sin(d_lat / 2.0)**2 + np.cos(rad_lat[:-1]) * np.cos(rad_lat[1:]) * np.sin(d_long / 2)**2
            c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

            distance = R * c
            velocity = ( distance*1000.0 )/( 60.0*np.array(d_time, dtype=float) )
            # Remove the first row
            self.data[key]= self.data[key].iloc[1:]
            self.data[key]["Velocity"] = pd.Series(velocity)
            
            # Drop columns with missing values
            self.data[key].dropna(axis=0, inplace=True)
            # Reset Index
            self.data[key].reset_index(inplace = True)

    def plot_trajectories(self, saving_path=None):
        
        def velocity_extremes(data):
            min_vel = np.inf
            max_vel = -np.inf

            for key in data.keys():
                min_vel_val = np.min(data[key]["Velocity"])
                max_vel_val = np.max(data[key]["Velocity"])

                min_vel = min_vel_val if min_vel_val < min_vel else min_vel
                max_vel = max_vel_val if max_vel_val > max_vel else max_vel

            return min_vel, max_vel
        
        def graph_centre(data):
            min_lat = 100
            max_lat = 0
            min_lon = 100
            max_lon = 0

            for key in data.keys():
                min_lat_val = np.min(data[key]["Latitude"])
                min_lon_val = np.min(data[key]["Longitude"])
                max_lat_val = np.max(data[key]["Latitude"])
                max_lon_val = np.max(data[key]["Longitude"])

                min_lat = min_lat_val if min_lat_val < min_lat else min_lat
                max_lat = max_lat_val if max_lat_val > max_lat else max_lat
                min_lon = min_lon_val if min_lon_val < min_lon else min_lon
                max_lon = max_lon_val if max_lon_val > max_lon else max_lon

            lat_centre = np.average([min_lat, max_lat])
            lon_centre= np.average([min_lon, max_lon])

            return lat_centre, lon_centre
        
        def plot_vel_traj(df_map, df, name, drog_depth, color_mapper):
            
            colors = ['#ffff00', '#ffffff', '#0000ff']
            drog_depths = np.unique(self.logsheet["DrogDepth"])
            
            drog_depth_found = False
            count = 0
            for depth in drog_depths:
                if drog_depth == depth:
                    drifter_col, drifter_info = [colors[count], f'{drog_depth}m Depth']
                    drog_depth_found = False
                    break
                count += 1
                
            if not drog_depth_found:
                drifter_col, drifter_info = ['#000000', 'Unknown']
                
            feature_group = folium.FeatureGroup(f"{name}\n{drifter_info}")
            
            folium.Marker([ df["Latitude"].iloc[-1]+0.001, df["Longitude"].iloc[-1]-0.001 ],
                    icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: blue">{f"{name}"}</div>""")
                    ).add_to(feature_group )
            
            start = False

            for lat, lon in zip(df["Latitude"], df["Longitude"]):
                if start: line = folium.PolyLine([[prev_lat, prev_long], [lat, lon]], color='white', weight=2).add_to(feature_group )
                else: start = True

                prev_lat = lat
                prev_long = lon


            for lat, lon, vel in zip(df["Latitude"], df["Longitude"], df["Velocity"]):
                circlemarker = folium.CircleMarker(location=(lat,lon), radius=6,
                                    stroke=True, color=drifter_col, weight=1.5, opacity=0.9,  
                                    fill=True, fill_color=color_mapper(vel)[:7], fill_opacity=0.9,
                                    popup=folium.Popup(f"{round(vel, 4)} m/s"),
                                ).add_to(feature_group )
            
            feature_group.add_to(df_map)

            return df_map
        
        def visualize_drifters(data, data_info):
            
            # Initializing the Map
            plot_map = folium.Map(location = graph_centre(data), zoom_start = 14.5, position="absolute", width='100%', height="100%", 
                                left='0%', top='0%', border=None, min_zoom=13, max_zoom=17)

            # position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;" allowfullscreen webkitallowfullscreen mozallowfullscreen

            # Plotting the location and velocities into the Map
            min_vel, max_vel = velocity_extremes(data)

            color_gradients = ['white', 'green', 'orange', 'red']
            velcolmap = bcm.LinearColormap(color_gradients, vmin=min_vel, vmax=max_vel, caption = 'Velocity (cm/s)') # The velocity scale
            velcolmap.add_to(plot_map)

            for key in data.keys():
                drog_depth = data_info[key]["DrogDepth"]
                plot_map = plot_vel_traj(plot_map, data[key], key, drog_depth, velcolmap)

            folium.LayerControl().add_to(plot_map)
            return plot_map
        
        self.map = visualize_drifters(self.data, self.data_info)
        
        if saving_path:
            self.map.save(f"{saving_path}/Graphs/TrajAndVelocity.html")
            
    def plot_boxplot(self):
        pass
    
    def plot_lineplot(self):
        pass

In [97]:
# Testing code
working_dir = "Data/2022/Day2"
D = Drifters("2022")
D.read_data(f"{working_dir}/drifters/")
D.read_logsheet(f"{working_dir}/drifters-logsheet.csv")

key = "274"

D.data[key]

,DateTime,Latitude,Longitude
0,2022-10-12 07:20:00,43.10376,5.91180
1,2022-10-12 07:30:00,43.08785,5.92787
2,2022-10-12 07:40:00,43.08125,5.95914
3,2022-10-12 07:50:00,43.07756,5.97256
4,2022-10-12 08:00:00,43.07716,5.97206
5,2022-10-12 08:10:00,43.07711,5.97130
6,2022-10-12 08:20:00,43.07718,5.97055
7,2022-10-12 08:30:00,43.07725,5.96965
8,2022-10-12 08:40:00,43.07736,5.96874
9,2022-10-12 08:50:00,43.07742,5.96792


In [98]:
D.logsheet

,Station,Name,DrogDepth,DepLong,DepLat,RecovLong,RecovLat,DepDateTime,RecovDateTime
0,1.0,274.0,1.0,5°58.363,43°4.655,5.0°57.564,43.0°4.783,2022-10-12 09:47:00,2022-10-12 12:40:00
1,1.0,8436.0,0.0,5°58.363,43°4.655,5.0°57.185,43.0°4.708,2022-10-12 09:47:00,2022-10-12 12:32:00
2,1.0,6439.0,0.6,5°58.363,43°4.655,5.0°57.571,43.0°4.803,2022-10-12 09:47:00,2022-10-12 12:42:00
3,2.0,273.0,1.0,5°59.055,43°4.723,5.0°57.743,43.0°5.145,2022-10-12 09:59:00,2022-10-12 14:03:00
4,2.0,7230.0,0.6,5°59.055,43°4.723,5.0°57.955,43.0°5.144,2022-10-12 09:59:00,2022-10-12 13:59:00
5,2.0,119.0,0.0,5°59.055,43°4.723,5.0°56.777,43.0°5.185,2022-10-12 09:59:00,2022-10-12 14:12:00
6,3.0,277.0,1.0,5°59.956,43°4.809,5.0°58.864,43.0°5.309,2022-10-12 10:16:00,2022-10-12 13:45:00
7,3.0,2052.0,0.0,5°59.956,43°4.809,5.0°58.161,43.0°5.361,2022-10-12 10:16:00,2022-10-12 13:52:00
8,3.0,3368.0,0.6,5°59.956,43°4.809,5.0°59.034,43.0°5.274,2022-10-12 10:16:00,2022-10-12 13:43:00


In [99]:
D.time_shift(2)

In [100]:
D.data[key].head()

,DateTime,Latitude,Longitude
0,2022-10-12 09:20:00,43.10376,5.91180
1,2022-10-12 09:30:00,43.08785,5.92787
2,2022-10-12 09:40:00,43.08125,5.95914
3,2022-10-12 09:50:00,43.07756,5.97256
4,2022-10-12 10:00:00,43.07716,5.97206


In [101]:
D.extract_data()
D.data[key].head()

C:\Users\olanr\AppData\Local\Temp\ipykernel_10968\2608569568.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[key].dropna(axis=0, inplace=True)


,index,DateTime,Latitude,Longitude
0,3,2022-10-12 09:50:00,43.07756,5.97256
1,4,2022-10-12 10:00:00,43.07716,5.97206
2,5,2022-10-12 10:10:00,43.07711,5.97130
3,6,2022-10-12 10:20:00,43.07718,5.97055
4,7,2022-10-12 10:30:00,43.07725,5.96965


In [102]:
D.time_shift(shift_amount=2)
D.data[key].head()

C:\Users\olanr\AppData\Local\Temp\ipykernel_10968\2608569568.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[key]["DateTime"] = self.data[key]["DateTime"] + timedelta(hours = shift_amount)


,index,DateTime,Latitude,Longitude
0,3,2022-10-12 11:50:00,43.07756,5.97256
1,4,2022-10-12 12:00:00,43.07716,5.97206
2,5,2022-10-12 12:10:00,43.07711,5.97130
3,6,2022-10-12 12:20:00,43.07718,5.97055
4,7,2022-10-12 12:30:00,43.07725,5.96965


In [103]:
D.compute_velocity()
D.data[key].head()

,level_0,index,DateTime,Latitude,Longitude,Velocity
0,1,4,2022-10-12 12:00:00,43.07716,5.97206,0.103328
1,2,5,2022-10-12 12:10:00,43.07711,5.97130,0.102383
2,3,6,2022-10-12 12:20:00,43.07718,5.97055,0.122558
3,4,7,2022-10-12 12:30:00,43.07725,5.96965,0.124899
4,5,8,2022-10-12 12:40:00,43.07736,5.96874,0.111592


In [104]:
D.plot_trajectories()
D.map

## 2022 Drifter Data Analysis

### Day 1

In [105]:
# Define the Day 1 Parameters
working_dir = "Data/2022/Day1"

# Create an Instance of the Drifter Class
D1 = Drifters("2022-1")

# Read the Data
D1.read_data(f"{working_dir}/drifters/")

# Read the Logsheet
D1.read_logsheet(f"{working_dir}/drifters-logsheet.csv")

# Shift time for the data
D1.time_shift(2)

# Extract Valid Data
D1.extract_data()

# Compute Velocity
D1.compute_velocity()

# Visualize the first 5 items of the modified data
D1.data['0119']

C:\Users\olanr\AppData\Local\Temp\ipykernel_10968\2608569568.py:142: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  bool = np.array([(deploy_time < pos_time < recov_time) for pos_time in self.data[key]["DateTime"]]).flatten()


,level_0,index,DateTime,Latitude,Longitude,Velocity
0,1,15,2022-10-11 13:47:46,43.07482,5.98532,0.000558
1,2,17,2022-10-11 13:42:48,43.07459,5.98482,0.000546
2,3,19,2022-10-11 13:37:47,43.07439,5.98431,0.000549
3,4,21,2022-10-11 13:32:47,43.07420,5.98379,0.000541
4,5,23,2022-10-11 13:27:49,43.07404,5.98326,0.000493
5,6,25,2022-10-11 13:23:25,43.07393,5.98276,0.000621
6,7,27,2022-10-11 13:17:52,43.07382,5.98212,0.000597
7,8,29,2022-10-11 13:12:51,43.07373,5.98150,0.000495
8,9,31,2022-10-11 13:07:51,43.07364,5.98099,0.000641
9,10,33,2022-10-11 13:02:52,43.07356,5.98032,0.000597


### Day 2

In [106]:
# Define the Day 1 Parameters
working_dir = "Data/2022/Day2"

# Create an Instance of the Drifter Class
D2 = Drifters("2022-2")

# Read the Data
D2.read_data(f"{working_dir}/drifters/")

# Read the Logsheet
D2.read_logsheet(f"{working_dir}/drifters-logsheet.csv")

# Shift time for the data
D2.time_shift(2)

# Extract Valid Data
D2.extract_data()

# Compute Velocity
D2.compute_velocity()

# Visualize the first 5 items of the modified data
D2.data['0119']

,level_0,index,DateTime,Latitude,Longitude,Velocity
0,1,1,2022-10-12 13:07:25,43.08342,5.95843,0.001133
1,2,2,2022-10-12 13:01:48,43.08297,5.95946,0.000856
2,3,3,2022-10-12 12:56:47,43.08281,5.96034,0.000823
3,4,4,2022-10-12 12:51:45,43.08254,5.96113,0.000861
4,5,5,2022-10-12 12:46:47,43.08228,5.96197,0.000939
5,6,6,2022-10-12 12:41:45,43.08194,5.96285,0.000939
6,7,7,2022-10-12 12:36:44,43.08163,5.96375,0.000873
7,8,8,2022-10-12 12:31:45,43.08129,5.96455,0.000911
8,9,9,2022-10-12 12:26:43,43.08100,5.96543,0.000944
9,10,10,2022-10-12 12:21:45,43.08068,5.96633,0.000950


### Day 2

In [107]:
# Define the Day 1 Parameters
working_dir = "Data/2022/Day3"

# Create an Instance of the Drifter Class
D3 = Drifters("2022-2")

# Read the Data
D3.read_data(f"{working_dir}/drifters/")

# Read the Logsheet
D3.read_logsheet(f"{working_dir}/drifters-logsheet.csv")

# Shift time for the data
D3.time_shift(2)

# Extract Valid Data
D3.extract_data()

# Compute Velocity
D3.compute_velocity()

# Visualize the first 5 items of the modified data
D3.data['0119']

,level_0,index,DateTime,Latitude,Longitude,Velocity
0,1,2,2022-10-13 14:20:41,43.08086,5.96910,0.000493
1,2,3,2022-10-13 14:16:06,43.08075,5.96860,0.000519
2,3,4,2022-10-13 14:10:40,43.08060,5.96809,0.000538
3,4,5,2022-10-13 14:05:41,43.08040,5.96759,0.000501
4,5,6,2022-10-13 14:00:42,43.08022,5.96712,0.000458
5,6,7,2022-10-13 13:55:40,43.08003,5.96671,0.000478
6,7,8,2022-10-13 13:50:39,43.07977,5.96635,0.000452
7,8,9,2022-10-13 13:45:41,43.07951,5.96603,0.000501
8,9,10,2022-10-13 13:40:40,43.07919,5.96573,0.000507
9,10,11,2022-10-13 13:35:41,43.07888,5.96540,0.000458
